In [8]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI


# Load environment variables from .env file
load_dotenv()


# Ensure the OpenAI API key is set in the environment
openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY environment variable is not set")


# Select the OpenAI model
model_name = "gpt-4o-mini"  # Replace with the desired model name

# Initialize the OpenAI LLM (Language Learning Model)
llm = ChatOpenAI(api_key=openai_api_key, model=model_name)

In [9]:
# Load docs
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./Basics of Writing Review Articles.pdf")
docs = loader.load()


# Split
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=25)
all_splits = text_splitter.split_documents(docs)


# Index
from langchain_community.vectorstores import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits
                                    , embedding=OpenAIEmbeddings()
                                    , persist_directory="data/review/vectorstore")
vectorstore.persist()

In [10]:
retriever = vectorstore.as_retriever(search_kwargs={'k':5})

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


# It takes in the documents as {context} and user provide {topic}
template = """Use the tips in the context:
{context} and edit the scientific review

Review: {review}


"""

prompt = ChatPromptTemplate.from_template(template)

model = llm

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    
# Using LangCHain LCEL to supply the prompt and generate output
chain = (
    {"context": retriever | format_docs, "review": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [12]:
import gradio as gr

def review_generator(review):
    story = chain.invoke(review)
    return story

demo = gr.Interface(
    fn=review_generator,
    inputs=["text"],
    outputs=["text"],
)

demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
